In [0]:
from pyspark.sql.functions import regexp_extract, trim, col, when, regexp_replace

# Load the existing invoice table
clean_df = spark.table("databricks_catalog.invoice_schema.customer_invoice_cleaned")

# Extract country from raw_text → into 'country_from_raw'
clean_df = clean_df.withColumn(
    "country_from_raw", 
    regexp_extract(
        col("raw_text"), 
        r"\b(Honduras|Vietnam|United States|France|Estonia|Austria|Mexico)\b", 
        1
    )
)

# Clean address newlines
clean_df = clean_df.withColumn("address_cleaned", regexp_replace("address", r"\n", " "))

# Extract country from address_cleaned → into 'country_from_address'
clean_df = clean_df.withColumn(
    "country_from_address",
    regexp_extract(col("address_cleaned"), r"(\b\w+(?: \w+)*?)\s*$", 1)
)

# Set 'country' column:
# - If address is empty, use country_from_raw
# - Else use country_from_address
clean_df = clean_df.withColumn(
    "country",
    when(
        (col("address").isNull()) | (trim(col("address")) == ""), 
        col("country_from_raw")
    ).otherwise(col("country_from_address"))
)

clean_df = clean_df.withColumn(
    "country",
    when(col("address_cleaned").rlike(r".*\bEngland\b.*"), "United Kingdom")
    .when(col("address_cleaned").rlike(r"^\d{6}"), "United States")
    .otherwise(col("country"))
)

# Trim leading/trailing spaces from the address column
clean_df = clean_df.withColumn("address", trim(col("address")))

# Drop rows where address is null or empty after trimming
clean_df = clean_df.filter((col("address").isNotNull()) & (col("address") != ""))

# Drop helper columns if desired
clean_df = clean_df.drop("country_from_raw", "country_from_address", "address_cleaned")

# Save the cleaned data back to a Delta table
clean_df.write.format("delta").mode("overwrite").saveAsTable(
    "databricks_catalog.invoice_schema.customer_invoice_country"
)

display(clean_df)
